# 4.4 Porównanie baggingu dla recesji cen

In [1]:
import pandas as pd

In [2]:
boston_df = pd.read_parquet("../data/boston_df.parquet")
boston_df.head(5)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [3]:
X = boston_df.drop(columns="MEDV")
y = boston_df["MEDV"]

## Bazowe drzewo decyzyjne

In [4]:
dt_best_params = {
    "ccp_alpha": 0.0,
    "criterion": "absolute_error",
    "max_depth": 7,
    "max_features": None,
    "min_samples_leaf": 8,
    "min_samples_split": 2,
}

In [5]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_validate, GridSearchCV

In [6]:
dt = DecisionTreeRegressor(**dt_best_params)
dt_cv_results = cross_validate(dt, X, y, scoring="neg_mean_squared_error", return_train_score=True)
dt_cv_results

{'fit_time': array([0.03388953, 0.01375222, 0.01329064, 0.01468062, 0.01476455]),
 'score_time': array([0.00142622, 0.00134277, 0.00131059, 0.00130558, 0.00121808]),
 'test_score': array([-10.83691176, -18.4255198 , -20.1259901 , -50.78074257,
        -28.67425743]),
 'train_score': array([-15.8024505 , -13.76149383, -14.75446914,  -7.80579012,
        -15.8002716 ])}

In [7]:
dt_cv_results["test_score"].mean()

-25.7686843331392

## Wykorzystanie Random Forest

In [8]:
from sklearn.ensemble import RandomForestRegressor

In [17]:
rf_params_grid = {
    "n_estimators": [2, 5, 15, 25, 50, 100],
}

rf_classifier = RandomForestRegressor(**rf_params_grid)
rf_cv = GridSearchCV(rf_classifier, param_grid=rf_params_grid,scoring="neg_mean_squared_error", cv=5, return_train_score=True)

In [18]:
rf_cv.fit(X, y)

GridSearchCV(cv=5,
             estimator=RandomForestRegressor(n_estimators=[2, 5, 15, 25, 50,
                                                           100]),
             param_grid={'n_estimators': [2, 5, 15, 25, 50, 100]},
             return_train_score=True, scoring='neg_mean_squared_error')

In [20]:
rf_cv.best_params_

{'n_estimators': 15}

In [21]:
rf_cv.best_score_

-21.401990453633598

## Porównanie z Extra Trees

In [22]:
from sklearn.ensemble import ExtraTreesRegressor

In [23]:
et_regressor = GridSearchCV(ExtraTreesRegressor(**dt_best_params), param_grid=rf_params_grid, scoring="neg_mean_squared_error", n_jobs=-1,
                            verbose=1, cv=5, return_train_score=True)
et_regressor.fit(X, y)

Fitting 5 folds for each of 6 candidates, totalling 30 fits


GridSearchCV(cv=5,
             estimator=ExtraTreesRegressor(criterion='absolute_error',
                                           max_depth=7, max_features=None,
                                           min_samples_leaf=8),
             n_jobs=-1, param_grid={'n_estimators': [2, 5, 15, 25, 50, 100]},
             return_train_score=True, scoring='neg_mean_squared_error',
             verbose=1)

In [24]:
et_regressor.best_score_

-22.530935104615605

In [25]:
et_regressor.best_params_

{'n_estimators': 100}